In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.metrics import accuracy_score, classification_report

In [7]:
data = np.load("../../Data/development.npy")
data_csv = pd.read_csv("../../Data/metadata/development.csv")
idx_to_feature_name = pd.read_csv("../../Data/metadata/idx_to_feature_name.csv")
labels = data_csv['word']
y = {}
labels_new = []
i = 0
for label in np.unique(labels).tolist():
    if label == 'other':
        continue
    else:
        y[label] = i
        i += 1
for label in labels:
    if label == 'other':
        continue
    else:
        labels_new.append(y[label])
data_no_other = data[:40834, :, :]
data_train = np.concatenate((data_no_other[:32667, :22, :], data_no_other[:32667, 87:, :]),
                                    axis=1)  # data[samples, features, ...]
data_valid = np.concatenate((data_no_other[32667:, :22, :], data_no_other[32667:, 87:, :]), axis=1)
print(data_train.shape, data_valid.shape)
labels_train = labels_new[:32667]
labels_valid = labels_new[32667:]

(32667, 110, 44) (8167, 110, 44)


In [6]:
X_train, X_valid, y_train, y_valid = data_train, data_valid, labels_train, labels_valid

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

# Set device to MPS if available
device = torch.device("mps" if torch.has_mps else "cpu")

# Define the function to train and evaluate the model
def train_and_evaluate(params):
    model = SimpleModel(input_dim=X_train.shape[1]).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])

    # Training loop
    model.train()
    for epoch in range(params['epochs']):
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device).float()
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        X_valid_device = X_valid.to(device)
        y_valid_device = y_valid.to(device).float()
        outputs = model(X_valid_device).squeeze()
        preds = (torch.sigmoid(outputs) > 0.5).int()
        accuracy = accuracy_score(y_valid_device.cpu(), preds.cpu())
        report = classification_report(y_valid_device.cpu(), preds.cpu())
    
    return accuracy, report

# Define parameter grid
param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'weight_decay': [0.0001, 0.001, 0.01],
    'epochs': [10, 20, 30]
}

# Perform grid search manually
best_params = None
best_accuracy = 0
best_report = None

for params in ParameterGrid(param_grid):
    accuracy, report = train_and_evaluate(params)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params
        best_report = report

print("Best parameters found: ", best_params)
print(f"Best Model Accuracy: {best_accuracy}")
print("Classification Report:")
print(best_report)

TypeError: 'int' object is not callable

In [8]:
X_train = data_train.reshape(data_train.shape[0], -1)
X_valid = data_valid.reshape(data_valid.shape[0], -1)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 3000],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'eta0': [0.001, 0.01, 0.1],
    'penalty': ['l2', 'l1', 'elasticnet']
}

# Initialize SGDClassifier
sgd_model = SGDClassifier(loss='hinge')

# Perform grid search
grid_search = GridSearchCV(sgd_model, param_grid, cv=5, n_jobs=-1, verbose=3)
grid_search.fit(X_train, labels_train)

# Best parameters and estimator
print("Best parameters found: ", grid_search.best_params_)
best_sgd_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_sgd_model.predict(X_valid)
accuracy = accuracy_score(labels_valid, y_pred)
report = classification_report(labels_valid, y_pred)

print(f"Best Model Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Fitting 5 folds for each of 432 candidates, totalling 2160 fits


KeyboardInterrupt: 

In [5]:


# Example data

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self, input_dim):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x)

# Set device to MPS if available
device = torch.device("mps" if torch.has_mps else "cpu")

# Define the function to train and evaluate the model
def train_and_evaluate(params):
    model = SimpleModel(input_dim=X_train.shape[1]).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])

    # Training loop
    model.train()
    for epoch in range(params['epochs']):
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device).float()
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        X_valid_device = X_valid.to(device)
        y_valid_device = y_valid.to(device).float()
        outputs = model(X_valid_device).squeeze()
        preds = (torch.sigmoid(outputs) > 0.5).int()
        accuracy = accuracy_score(y_valid_device.cpu(), preds.cpu())
        report = classification_report(y_valid_device.cpu(), preds.cpu())
    
    return accuracy, report

# Define parameter grid
param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'weight_decay': [0.0001, 0.001, 0.01],
    'epochs': [10, 20, 30]
}

# Perform grid search manually
best_params = None
best_accuracy = 0
best_report = None

for params in ParameterGrid(param_grid):
    accuracy, report = train_and_evaluate(params)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params
        best_report = report

print("Best parameters found: ", best_params)
print(f"Best Model Accuracy: {best_accuracy}")
print("Classification Report:")
print(best_report)


NameError: name 'X' is not defined